# Crawler
This notebook contains started code structure for creating a crawler on single machine

Author: Noshaba Nasir

Date: 26/3/2021

Updated by:

In [190]:
import os 
import random
from queue import Queue
from urllib.parse import urlparse

import urllib.request
from bs4 import BeautifulSoup
import ssl
import urllib.robotparser as rb
from urllib.request import urlopen
import requests

ssl._create_default_https_context = ssl._create_unverified_context
# Add any library to be imported here

In [2]:
# Crawler Parameters
BACKQUEUES= 3
THREADS= BACKQUEUES*3
FRONTQUEUES= 5
WAITTIME= 15 ; # wait 15 seconds before fetching URLS from 

# Add any other global parameters here

# FRONTIER
Frontier should use the Mercator frontier design as discussed in lecture.

Preferably it should be a class and should have the given functions.

prioritizer function is a stub right now, it will return a random number between 1 to f for given URL# 

In [3]:
class frontier:
# add the code for frontier here
# should have functions __init__, get_URL, add_URLs, add_to_backqueue
    FrontQueue = []
    BackQueue = []
    def __init__(self):
        self.FrontQueue = []
        self.BackQueue = []
        self.dic = {}  #dic is a table to maintain the domain name maintained by each backqueue
        for i in range(FRONTQUEUES):
            self.FrontQueue.append(Queue())
        for j in range(BACKQUEUES):
            self.BackQueue.append(Queue())
        
        for k in range(BACKQUEUES):
            self.dic[k] = None
       
        
    def add_URLs_into_frontqueue(self,url):
        rand = prioritizer(url,FRONTQUEUES)
        self.FrontQueue[rand-1].put(url)
    def get_URL_from_frontqueue(self):
        queue = selected_queue()
        print(str(queue))
        print("before while")
        while self.FrontQueue[queue].empty()==True:
            queue = selected_queue()
        print("after while")
        u = self.FrontQueue[queue].get()
        print(u)
        return u
    def add_to_backqueue(self,queuenumber,url):
        self.BackQueue[queuenumber].put(url)
    
    def check_backqueue_exists(self,url): #check if a backqueue already exists for a url,return the queue number otherwise -1
        backqueue_number = -1
        url_domain = urlparse(url).netloc #domain of the url
        
        for i in range(BACKQUEUES):
            if self.dic[i]==url_domain:
                return i
        return -1

        
        

def prioritizer(URL,f):
    """
    Take URL and returns priority from 1 to F
    Right now it like a stub function. 
    It will return a random number from 1 to f for given inputs. 
    """
    return random.randint(1,f)

In [4]:
#function to select the queue from which the url will be extracted to put into the backqueue
def selected_queue():
    prob = random.randint(1,100)
    queue = -1
    if prob<=30:
        queue = 0
    elif prob<=55:
        queue = 1
    elif prob<=75:
        queue = 2
    elif prob<=90:
        queue = 3
    else:
        queue = 4
    return queue

In [5]:
#heap for backqueue
#using list for now instead of heap
hp = {}
for i in range(BACKQUEUES):
    hp[i] = None
def extract_min_from_hp():
    #min_time=""
    #min_queue_number = -1
    #for i in hp:
    #    if hp[i] < min_time:
    #        min_time = hp[i]
    #        min_queue_number = i
    #return min_time,min_queue_number
    
    #for now just return a random number
    return random.randint(0,BACKQUEUES-1)
    


# FILTER URLS
Filter the URLS that are in robots.txt files of server and the have been already processed.

In [133]:
ua = 'User-agent' 
# write code for filtering urls here
#if in robot.txt
#or already visited
def robot_url(url):
    domain_url = '{uri.scheme}://{uri.netloc}'.format(uri=urlparse(url))
    robots_url = domain_url + '/robots.txt'
    
    return robots_url

def read_robot(url):
    roboturl = robot_url(url)
    
    txt = os.popen(f'curl {roboturl}').read()
    print(txt)
    return txt
   
def initialize_dict(url):
    robot_file = read_robot(url)
    print(robot_file)
    result_data_set = {ua:{}}
    for line in robot_file.split("\n"):
        if line.startswith(ua):
            result_data_set[ua].update({line.split(':')[1].strip():{}})
    keys = []
    for key in result_data_set[ua].keys():
        keys.append(key)
    return result_data_set, keys, robot_file
 


In [134]:
result_data_set, keys, robot_file = initialize_dict("https://en.wikipedia.org/wiki/Machine_learning")
keys

UnicodeDecodeError: 'charmap' codec can't decode byte 0x9d in position 7518: character maps to <undefined>

In [147]:


def filter_lib(url):
    
    bot = rb.RobotFileParser()
    u = robot_url(url) 
    print(u)
    # checks where the website's robot.txt file reside
    x = bot.set_url(u)
    y = bot.read()
    # we can crawl the main site
    z = bot.can_fetch('*', url)
    return z


#filter_lib("https://www.csie.ntu.edu.tw../index.html")

In [135]:
def is_url(url):
  try:
    result = urlparse(url)
    return all([result.scheme, result.netloc])
  except ValueError:
    return False

In [136]:
is_url("https://www.csie.ntu.edu.tw../index.html")

True

# Run Crawler

In [7]:
seen_urls={}

#helper_functions
def is_absolute(url): #checks if the given url is absolute or relative
    return bool(urlparse(url).netloc)

In [184]:
# Theard task
def crawler_thread_task():
    urls_extracted=[]
    print("into the thread")
    back_queue = extract_min_from_hp()
    print("backqueue= " + str(back_queue))
    if Frontier.BackQueue[back_queue].empty()==True:
        #print("found empty back queue")
        
        url = Frontier.get_URL_from_frontqueue()
        #print("url extracted from front queue= " + url)
        queue_number = Frontier.check_backqueue_exists(url)
        print("url found belonging to a domain? " + str(queue_number))
        found = False #till a new domain is found for which there is no back_queue
        while found==False:
            
            if(queue_number!=-1):
                #print("queue_number = " + str(queue_number))
                Frontier.add_to_backqueue(queue_number,url)
                url = Frontier.get_URL_from_frontqueue()
                queue_number = Frontier.check_backqueue_exists(url)
            else:
                #print("found new domain")
                #SHOULD READ THE URL OR NOT????
                url_domain = urlparse(url).netloc #domain of the url
                Frontier.dic[back_queue] = url_domain
                Frontier.BackQueue[back_queue].put(url)
                found=True

    
    else:
        u = Frontier.BackQueue[back_queue].get()
        print("PROBLEM U: " + u)
        b = 200
        try:
            a = urllib.request.urlopen(u)
        except urllib.error.HTTPError as e:
            b = 404
            
        except urllib.error.URLError as e:
            b = 400
        except urllib.error.UnicodeError as e:
            b = 400
       
        print("bruhhhhh b =" + str(b))
        
        if is_absolute(u)==True : #checking here if the url is valid
          
            seen_urls[u] = u #marking in the dic that the url is seen now


            url_domain2 = urlparse(u).netloc #extract the domain to convert it into absolute if relative
            shm=urlparse(u).scheme
            print(url_domain2)
            #got the url no go to that url see its content save the content put it into the seen urls and look for more urls
            with urllib.request.urlopen(u) as response:
                html = response.read()
                #ADD CHECK IF THE PAGE IS HTML!!
                soup = BeautifulSoup(html, 'html.parser')
                #print(soup.prettify())
                
                for link in soup.find_all('a'):
                    a = link.get('href')
                    if a is not None:
                        #print("Not Nonee")
                        #if a.find("!DOCTYPE html")!=-1:
                        #print("YAYY")
                        if is_absolute(a)==False:
                            #CHECK IF HTML
                            a = shm + "://" + url_domain2 + a 
                            
                            urls_extracted.append(a)
                for item in urls_extracted:
                    print (item)
    return urls_extracted



In [185]:
# intialize every thing
hp = {}
Frontier = frontier()
#Adding seed urls in frontqueues
f = open("seed.txt", "r")
seed_urls = f.read()
urls = seed_urls.split("\n")
for url in urls:
    Frontier.add_URLs_into_frontqueue(url)

for i in range(FRONTQUEUES):
    print("for queue" + str(i))
    print(Frontier.FrontQueue[i].qsize())

for queue0
2
for queue1
3
for queue2
2
for queue3
1
for queue4
0


In [186]:

# start the threads
for i in range(100):
    urls = crawler_thread_task()
    for item in urls:
        Frontier.add_URLs_into_frontqueue(item)
        


into the thread
backqueue= 0
2
before while
after while
https://en.wikipedia.org/wiki/Machine_learning
url found belonging to a domain? -1
into the thread
backqueue= 2
2
before while
after while
https://en.wikipedia.org/api/
url found belonging to a domain? 0
0
before while
after while
https://www.csie.ntu.edu.tw/~cjlin/libsvm/index.html
into the thread
backqueue= 0
PROBLEM U: https://en.wikipedia.org/wiki/Machine_learning
bruhhhhh b =200
en.wikipedia.org
https://en.wikipedia.org#mw-head
https://en.wikipedia.org#searchInput
https://en.wikipedia.org/wiki/Machine_Learning_(journal)
https://en.wikipedia.org/wiki/Statistical_learning_in_language_acquisition
https://en.wikipedia.org/wiki/Data_mining
https://en.wikipedia.org/wiki/File:Multi-Layer_Neural_Network-Vector-Blank.svg
https://en.wikipedia.org/wiki/Statistical_classification
https://en.wikipedia.org/wiki/Cluster_analysis
https://en.wikipedia.org/wiki/Regression_analysis
https://en.wikipedia.org/wiki/Anomaly_detection
https://en.wiki

bruhhhhh b =200
www.csie.ntu.edu.tw
https://www.csie.ntu.edu.tw../index.html
https://www.csie.ntu.edu.tw../libsvmtools
https://www.csie.ntu.edu.tw../libsvmtools/datasets
https://www.csie.ntu.edu.tw../papers/guide/guide.pdf
https://www.csie.ntu.edu.tw../papers/guide/guide.pdf
https://www.csie.ntu.edu.tw#nuandone
https://www.csie.ntu.edu.tw#nuandone
https://www.csie.ntu.edu.tw#nuandone
https://www.csie.ntu.edu.tw../papers/quadworkset.pdf
https://www.csie.ntu.edu.twfaq.html#f203
https://www.csie.ntu.edu.tw#java
https://www.csie.ntu.edu.tw#GUI
https://www.csie.ntu.edu.tw#python
https://www.csie.ntu.edu.tw#R
https://www.csie.ntu.edu.tw#matlab
https://www.csie.ntu.edu.tw#perl
https://www.csie.ntu.edu.tw#ruby
https://www.csie.ntu.edu.tw#weka
https://www.csie.ntu.edu.tw#lisp
https://www.csie.ntu.edu.tw#clisp
https://www.csie.ntu.edu.tw#haskell
https://www.csie.ntu.edu.tw#ocaml
https://www.csie.ntu.edu.tw#labview
https://www.csie.ntu.edu.tw#PHP
https://www.csie.ntu.edu.tw#csharp
https://www.csi

before while
after while
https://en.wikipedia.org#cite_ref-64
3
before while
after while
https://en.wikipedia.org/w/index.php?title=Machine_learning&action=edit&section=25
1
before while
after while
https://en.wikipedia.org/wiki/Machine_Learning_(journal)
3
before while
after while
https://en.wikipedia.org/wiki/Support-vector_machine
1
before while
after while
https://en.wikipedia.org/wiki/Machine_Learning_(journal)
0
before while
after while
https://en.wikipedia.org#cite_ref-75
3
before while
after while
https://en.wikipedia.org#cite_note-CorinnaCortes-69
0
before while
after while
https://en.wikipedia.org/wiki/Chartered_Financial_Analyst_(CFA)#Curriculum
2
before while
after while
https://en.wikipedia.org#cite_ref-32
0
before while
after while
https://en.wikipedia.org/wiki/The_Wall_Street_Journal
1
before while
after while
https://en.wikipedia.org#cite_ref-76
3
before while
after while
https://en.wikipedia.org/wiki/Logistic_regression
1
before while
after while
https://en.wikipedia.o

UnicodeError: encoding with 'idna' codec failed (UnicodeError: label empty or too long)

In [33]:
def is_absolute(url): #checks if the given url is absolute or relative
    return bool(urlparse(url).netloc)
url = "https://en.wikipedia.org/w/api.php"
with urllib.request.urlopen(url) as response:
   html = response.read()
#ADD CHECK IF THE PAGE IS HTML!!
soup = BeautifulSoup(html, 'html.parser')
myfile = open("items2.xml", "w")
myfile.write(str(soup))
#print(soup.prettify())

#url_domain = urlparse(url).netloc #extract the domain to convert it into absolute if relative
#list=[]
#for link in soup.find_all('a'):
#    a = link.get('href')
#    if is_absolute(a)==False:
#        a = "https://" + url_domain + a        
#    list.append(a)
#for item in list:
#    print (item)

58990

In [ ]:
def is_absolute(url):
    return bool(urlparse(url).netloc)
web_url = 'https:www.example-page-xl.com'
sauce = urllib.request.urlopen(web_url).read()
soup = bs.BeautifulSoup(sauce,'lxml')

section = soup.section

for url in section.find_all('a'):
    print urllib.parse.urljoin(web_url,url.get('href'))

In [ ]:
def is_absolute(url):
    return bool(urlparse(url).netloc)

url="https://en.wikipedia.org/w/api.php"
url_domain = urlparse(url).netloc
print()

In [ ]:


            with urllib.request.urlopen(u) as response:
                html = response.read()
                #ADD CHECK IF THE PAGE IS HTML!!
                soup = BeautifulSoup(html, 'html.parser')
                #print(soup.prettify())
                urls_extracted=[]
                for link in soup.find_all('a'):
                    a = link.get('href')
                    if is_absolute(a)==False:
                        a = "https://" + url_domain + a 
                    urls_extracted.append(a)
                    for item in list:
                        print (item)

In [ ]:
print(Frontier.get_URL_from_frontqueue())

In [ ]:
print(selected_queue())

In [ ]:
Frontier.FrontQueue[2].qsize()

In [199]:
b = 200
try:
    url = 'https://www.csie.ntu.edu.tw../index.html'
    request = requests.get(url)
except UnicodeError as e:
    request.status_code = 404
            

print(request.status_code)

AttributeError: module 'urllib' has no attribute 'urlopen'

### 